In [3]:
import numpy as np
import cv2

#We're using function from image_recognition to process photos
# from image_recognition import

#This is where the classification and categorization of images will occur
data_dir = 'Dataset'
train = 'Dataset/training'
test = 'Dataset/evaluation'
valid = 'Dataset/validation'
label_names = ['Bread', 'Dairy Product', 'Dessert', 'Egg', 'Fried Food', 'Meat', 'Noodles-Pasta', 'Rice',
               'Seafood', 'Soup', 'Vegetable-Fruit']
training_set_size = [10,100, 250, 500, 1000, 5000, 9000]
test_set_size = [10, 100, 250, 500, 1000, 2000, 3000]
valid_set_size = [10, 100, 250, 500, 1000, 2000, 3000]

In [4]:
def train_one_vs_all(X, y, num_classes, lambda_val):
    '''
    Train a one vs. all logistic regression
    
    Inputs: 
      X                data matrix (2d array shape m x n)
      y                label vector with entries from 0 to 
                       num_classes - 1 (1d array length m)
      num_classes      number of classes (integer)
      lambda_val       regularization parameter (scalar)

    Outputs:
      weight_vectors   matrix of weight vectors for each class 
                       weight vector for class c in the cth column
                       (2d array shape n x num_classes)
      intercepts       vector of intercepts for all classes
                       (1d array length num_classes)                       
            
    '''
    #Hint: you may find the vector comparison y == i helpful!
    #Y is gonna be some length (between 0-9), so for each class build a new list and go through each possible number.
    #Go through the Y list, compare it to the current number and if it matches then make it 1 (if not then a 0). Shove that new number into a new list (that's the same size of Y, but conly composed of 1s and 0s)
    weight_vectors = []
    intercepts = []

In [5]:
def predict_one_vs_all(X, weight_vectors, intercepts):
    predictions = []
    return predictions

In [15]:
import os
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory( 
    directory=train, 
    classes = label_names,
    target_size=(224, 224),  
    batch_size=32, 
    class_mode="binary", 
)
# validation data
valid_generator = datagen.flow_from_directory( 
    directory=valid, 
    classes = label_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary", 
)

resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in resnet_50.layers:
    layer.trainable = False
# build the entire model
x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(11, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

# define training function
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=epochs, batch_size=batch_size)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [16]:
model= trainModel(model, 10, "Adam")

Epoch 1/10
309/309 [==============================] - 620s 2s/step - loss: 2.3175 - accuracy: 0.1816 - val_loss: 1.9885 - val_accuracy: 0.3146
Epoch 2/10
309/309 [==============================] - 414s 1s/step - loss: 1.8887 - accuracy: 0.3347 - val_loss: 1.5392 - val_accuracy: 0.5271
Epoch 3/10
309/309 [==============================] - 411s 1s/step - loss: 1.6352 - accuracy: 0.4478 - val_loss: 1.2779 - val_accuracy: 0.6236
Epoch 4/10
309/309 [==============================] - 414s 1s/step - loss: 1.4618 - accuracy: 0.5185 - val_loss: 1.1910 - val_accuracy: 0.6525
Epoch 5/10
113/309 [=========>....................] - ETA: 3:15 - loss: 1.3689 - accuracy: 0.5604